# Importing all the required libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
import plotly
from pathlib import Path
from IPython.display import display
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sahik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
data = pd.read_csv('C:\\Users\\sahik\\Desktop\\data.csv',encoding='unicode_escape')

data.shape

(541909, 8)

In [4]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [6]:
tab_info=pd.DataFrame(data.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()).T.rename(index={0:'null values (nb)'}))

In [7]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
data.dropna(axis = 0, inplace = True)
print(data.shape)   

(406829, 8)


In [9]:
print('Duplicate Entries: {}'.format(data.duplicated().sum()))
data.drop_duplicates(inplace = True)

Duplicate Entries: 5225


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.6+ MB


In [11]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


### Categories of the product

In [12]:
pd.DataFrame([{'products': len(data['StockCode'].value_counts()),'transactions': len(data['InvoiceNo'].value_counts()),
               'customers': len(data['CustomerID'].value_counts()),  
              }], columns = ['products', 'transactions', 'customers'], index = ['quantity'])

,products,transactions,customers
quantity,3684,22190,4372


In [13]:
tp = data.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
no_products = tp.rename(columns = {'InvoiceDate':'Number of products'})
no_products[:10].sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of products
0,12346.0,541431,1
1,12346.0,C541433,1
2,12347.0,537626,31
3,12347.0,542237,29
4,12347.0,549222,24
5,12347.0,556201,18
6,12347.0,562032,22
7,12347.0,573511,47
8,12347.0,581180,11
9,12348.0,539318,17


In [14]:
no_products['order_canceled'] = no_products['InvoiceNo'].apply(lambda x:int('C' in x))
display(no_products[:5])

n_1 = no_products['order_canceled'].sum()
n_2 = no_products.shape[0]


,CustomerID,InvoiceNo,Number of products,order_canceled
0,12346.0,541431,1,0
1,12346.0,C541433,1,1
2,12347.0,537626,31,0
3,12347.0,542237,29,0
4,12347.0,549222,24,0


In [15]:
data.sort_values('CustomerID')[:5]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
286623,562032,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2011-08-02 08:48:00,4.25,12347.0,Iceland
72260,542237,84991,60 TEATIME FAIRY CAKE CASES,24,2011-01-26 14:30:00,0.55,12347.0,Iceland
14943,537626,22772,PINK DRAWER KNOB ACRYLIC EDWARDIAN,12,2010-12-07 14:57:00,1.25,12347.0,Iceland


In [16]:
cleaned = data.copy(deep = True)
cleaned['QuantityCanceled'] = 0

remove = [] ; entry = []

for index, col in  data.iterrows():
    if (col['Quantity'] > 0) or col['Description'] == 'Discount': continue        
    dtest = data[(data['CustomerID'] == col['CustomerID']) &
                         (data['StockCode']  == col['StockCode']) & 
                         (data['InvoiceDate'] < col['InvoiceDate']) & 
                         (data['Quantity']   > 0)].copy()
    
    
    if (dtest.shape[0] == 0): 
        entry.append(index)
  

    elif (dtest.shape[0] == 1): 
        index_order = dtest.index[0]
        data.loc[index_order, 'QuantityCanceled'] = -col['Quantity']
        remove.append(index)        

        
    elif (dtest.shape[0] > 1): 
        dtest.sort_index(axis=0 ,ascending=False, inplace = True)        
        for ind, val in dtest.iterrows():
            if val['Quantity'] < -col['Quantity']: continue
            cleaned.loc[ind, 'QuantityCanceled'] = -col['Quantity']
            remove.append(index) 
            break     

In [17]:
len(remove)
len(entry)
cleaned.drop(remove, axis = 0, inplace = True)
cleaned.drop(entry, axis = 0, inplace = True)
remaining = cleaned[(cleaned['Quantity'] < 0) & (cleaned['StockCode'] != 'D')]

In [18]:
format(remaining.shape[0])

'48'

In [19]:
listcodes = cleaned[cleaned['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
listcodes

array(['POST', 'D', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT'],
      dtype=object)

In [20]:
cleaned['TotalPrice'] = cleaned['UnitPrice'] * (cleaned['Quantity'] - cleaned['QuantityCanceled'])
cleaned.sort_values('CustomerID')[:5]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,QuantityCanceled,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,0,77183.6
148288,549222,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2011-04-07 10:43:00,4.25,12347.0,Iceland,0,17.0
428971,573511,22698,PINK REGENCY TEACUP AND SAUCER,12,2011-10-31 12:25:00,2.95,12347.0,Iceland,0,35.4
428970,573511,47559B,TEA TIME OVEN GLOVE,10,2011-10-31 12:25:00,1.25,12347.0,Iceland,0,12.5
428969,573511,47567B,TEA TIME KITCHEN APRON,6,2011-10-31 12:25:00,5.95,12347.0,Iceland,0,35.7


In [21]:
tp = cleaned.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['TotalPrice'].sum()
basket = tp.rename(columns = {'TotalPrice':'Basket Price'})


cleaned['InvoiceDate_int'] = cleaned['InvoiceDate'].astype('int64')
tp = cleaned.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate_int'].mean()
cleaned.drop('InvoiceDate_int', axis = 1, inplace = True)
basket.loc[:, 'InvoiceDate'] = pd.to_datetime(tp['InvoiceDate_int'])


basket = basket[basket['Basket Price'] > 0]
basket.sort_values('CustomerID')[:6]

,CustomerID,InvoiceNo,Basket Price,InvoiceDate
0,12346.0,541431,77183.60,2011-01-18 10:01:00.000000000
1,12347.0,537626,711.79,2010-12-07 14:57:00.000001024
2,12347.0,542237,475.39,2011-01-26 14:29:59.999999744
3,12347.0,549222,636.25,2011-04-07 10:42:59.999999232
4,12347.0,556201,382.52,2011-06-09 13:01:00.000000256
5,12347.0,562032,584.91,2011-08-02 08:48:00.000000000


In [22]:
noun = lambda pos: pos[:2] == 'NN'

def keywords_inventory(dataframe, colonne = 'Description'):
    stemmer = nltk.stem.SnowballStemmer("english")
    keywords_roots  = dict() 
    keywords_select = dict() 
    category_keys   = []
    count_keywords  = dict()
    icount = 0
    for s in dataframe[colonne]:
        if pd.isnull(s): continue
        lines = s.lower()
        tokenized = nltk.word_tokenize(lines)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if noun(pos)] 
        
        for t in nouns:
            t = t.lower() ; racine = stemmer.stem(t)
            if racine in keywords_roots:                
                keywords_roots[racine].add(t)
                count_keywords[racine] += 1                
            else:
                keywords_roots[racine] = {t}
                count_keywords[racine] = 1
    
    for s in keywords_roots.keys():
        if len(keywords_roots[s]) > 1:  
            min_length = 1000
            for k in keywords_roots[s]:
                if len(k) < min_length:
                    clef = k ; min_length = len(k)            
            category_keys.append(clef)
            keywords_select[s] = clef
        else:
            category_keys.append(list(keywords_roots[s])[0])
            keywords_select[s] = list(keywords_roots[s])[0]
                   
    print("variable '{}': {}".format(colonne,len(category_keys)))
    return category_keys, keywords_roots, keywords_select, count_keywords

In [23]:
products = pd.DataFrame(data['Description'].unique()).rename(columns = {0:'Description'})

In [24]:
keywords, keywords_roots, keywords_select, count_keywords = keywords_inventory(products)

variable 'Description': 1484


In [25]:
l_products = []
for k,v in count_keywords.items():
    l_products.append([keywords_select[k],v])
l_products.sort(key = lambda x:x[1], reverse = True)

In [26]:
l_products = []
for k,v in count_keywords.items():
    word = keywords_select[k]
    if word in ['pink', 'blue', 'tag', 'green', 'orange']: continue
    if len(word) < 3 or v < 13: continue
    if ('+' in word) or ('/' in word): continue
    l_products.append([word, v])

l_products.sort(key = lambda x:x[1], reverse = True)
len(l_products)

193

In [27]:
produit = cleaned['Description'].unique()
X = pd.DataFrame()
for key, occurence in l_products:
    X.loc[:, key] = list(map(lambda x:int(key.upper() in x), produit))

In [28]:
threshold = [0, 1, 2, 3, 5, 10]
label_col = []
for i in range(len(threshold)):
    if i == len(threshold)-1:
        col = '.>{}'.format(threshold[i])
    else:
        col = '{}<.<{}'.format(threshold[i],threshold[i+1])
    label_col.append(col)
    X.loc[:, col] = 0

for i, prod in enumerate(produit):
    prix = cleaned[ cleaned['Description'] == prod]['UnitPrice'].mean()
    j = 0
    while prix > threshold[j]:
        j+=1
        if j == len(threshold): break
    X.loc[i, label_col[j-1]] = 1

In [29]:
print("{:<8} {:<20} \n".format('range', 'Products') + 20*'-')
for i in range(len(threshold)):
    if i == len(threshold)-1:
        col = '.>{}'.format(threshold[i])
    else:
        col = '{}<.<{}'.format(threshold[i],threshold[i+1])    
    print("{:<10}  {:<20}".format(col, X.loc[:, col].sum()))

range    Products             
--------------------
0<.<1       964                 
1<.<2       1009                
2<.<3       673                 
3<.<5       606                 
5<.<10      470                 
.>10        156                 


In [30]:
matrix = X.values
for n_clusters in range(2,10):
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

For n_clusters = 2 The average silhouette_score is : 0.06999283177793626
For n_clusters = 3 The average silhouette_score is : 0.10071681758064248
For n_clusters = 4 The average silhouette_score is : 0.11986892180532537
For n_clusters = 5 The average silhouette_score is : 0.1466257603527048
For n_clusters = 6 The average silhouette_score is : 0.14389114353988738
For n_clusters = 7 The average silhouette_score is : 0.14979240620319792
For n_clusters = 8 The average silhouette_score is : 0.15070097448972056
For n_clusters = 9 The average silhouette_score is : 0.14939568026429534


In [31]:
n_clusters = 5
silhouette_avg = -1
while silhouette_avg < 0.145:
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters)
    silhouette_avg

In [32]:
pd.Series(clusters).value_counts()

2    1009
3     964
1     829
0     606
4     470
dtype: int64

In [33]:
liste = pd.DataFrame(produit)
liswords = [word for (word, occurence) in l_products]

occurence = [dict() for _ in range(n_clusters)]

for i in range(n_clusters):
    l_cluster = liste.loc[clusters == i]
    for word in liswords:
        if word in ['art', 'set', 'heart', 'pink', 'blue', 'tag']: continue
        occurence[i][word] = sum(l_cluster.loc[:, 0].str.contains(word.upper()))

In [34]:
pca = PCA()
pca.fit(matrix)
pca_samples = pca.transform(matrix)

In [35]:
pca = PCA(n_components=50)
matrix_9D = pca.fit_transform(matrix)
mat = pd.DataFrame(matrix_9D)
mat['cluster'] = pd.Series(clusters)

In [36]:
corresp = dict()
for key, val in zip (produit, clusters):
    corresp[key] = val
cleaned['categ_product'] = cleaned.loc[:, 'Description'].map(corresp)
for i in range(5):
    col = 'categ_{}'.format(i)        
    df = cleaned[cleaned['categ_product'] == i]
    price_temp = df['UnitPrice'] * (df['Quantity'] - df['QuantityCanceled'])
    price_temp = price_temp.apply(lambda x:x if x > 0 else 0)
    cleaned.loc[:, col] = price_temp
    cleaned[col].fillna(0, inplace = True)

cleaned[['InvoiceNo', 'Description', 'categ_product', 'categ_0', 'categ_1', 'categ_2', 'categ_3','categ_4']][:15]

,InvoiceNo,Description,categ_product,categ_0,categ_1,categ_2,categ_3,categ_4
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,1,0.00,15.3,0.00,0.0,0.0
1,536365,WHITE METAL LANTERN,0,20.34,0.0,0.00,0.0,0.0
2,536365,CREAM CUPID HEARTS COAT HANGER,0,22.00,0.0,0.00,0.0,0.0
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,0,20.34,0.0,0.00,0.0,0.0
4,536365,RED WOOLLY HOTTIE WHITE HEART.,0,20.34,0.0,0.00,0.0,0.0
5,536365,SET 7 BABUSHKA NESTING BOXES,4,0.00,0.0,0.00,0.0,15.3
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER,0,25.50,0.0,0.00,0.0,0.0
7,536366,HAND WARMER UNION JACK,1,0.00,11.1,0.00,0.0,0.0
8,536366,HAND WARMER RED POLKA DOT,2,0.00,0.0,11.10,0.0,0.0
9,536367,ASSORTED COLOUR BIRD ORNAMENT,2,0.00,0.0,54.08,0.0,0.0


In [37]:
tp = cleaned.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['TotalPrice'].sum()
basket_p = tp.rename(columns = {'TotalPrice':'Basket Price'})


for i in range(5):
    col = 'categ_{}'.format(i) 
    tp = cleaned.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)[col].sum()
    basket_p.loc[:, col] = tp 

cleaned['InvoiceDate_int'] = cleaned['InvoiceDate'].astype('int64')
tp = cleaned.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate_int'].mean()
cleaned.drop('InvoiceDate_int', axis = 1, inplace = True)
basket_p.loc[:, 'InvoiceDate'] = pd.to_datetime(tp['InvoiceDate_int'])

basket_p = basket_p[basket_p['Basket Price'] > 0]
basket_p.sort_values('CustomerID', ascending = True)[:5]

,CustomerID,InvoiceNo,Basket Price,categ_0,categ_1,categ_2,categ_3,categ_4,InvoiceDate
0,12346.0,541431,77183.60,0.00,0.00,77183.6,0.00,0.00,2011-01-18 10:01:00.000000000
1,12347.0,537626,711.79,293.35,83.40,187.2,23.40,124.44,2010-12-07 14:57:00.000001024
2,12347.0,542237,475.39,169.20,91.35,130.5,84.34,0.00,2011-01-26 14:29:59.999999744
3,12347.0,549222,636.25,115.00,109.35,330.9,81.00,0.00,2011-04-07 10:42:59.999999232
4,12347.0,556201,382.52,168.76,78.06,74.4,41.40,19.90,2011-06-09 13:01:00.000000256


In [38]:
basket_p['InvoiceDate'].min(), '->',  basket_p['InvoiceDate'].max()

(Timestamp('2010-12-01 08:26:00'), '->', Timestamp('2011-12-09 12:50:00'))

In [39]:
transactions=basket_p.groupby(by=['CustomerID'])['Basket Price'].agg(['count','min','max','mean','sum'])
for i in range(5):
    col = 'categ_{}'.format(i)
    transactions.loc[:,col] = basket_p.groupby(by=['CustomerID'])[col].sum() /\
                                            transactions['sum']*100

transactions.reset_index(drop = False, inplace = True)
basket_p.groupby(by=['CustomerID'])['categ_0'].sum()
transactions.sort_values('CustomerID', ascending = True)[:6]

,CustomerID,count,min,max,mean,sum,categ_0,categ_1,categ_2,categ_3,categ_4
0,12346.0,1,77183.60,77183.60,77183.600000,77183.60,0.000000,0.000000,100.000000,0.000000,0.000000
1,12347.0,7,224.82,1294.32,615.714286,4310.00,29.540371,25.242459,26.375870,11.237123,7.604176
2,12348.0,4,227.44,892.80,449.310000,1797.24,0.000000,20.030714,41.953217,38.016069,0.000000
3,12349.0,1,1757.55,1757.55,1757.550000,1757.55,10.713778,37.877728,26.506216,4.513101,20.389178
4,12350.0,1,334.40,334.40,334.400000,334.40,0.000000,39.862440,48.444976,11.692584,0.000000
5,12352.0,8,100.53,840.30,310.780000,2486.24,14.608405,52.954662,17.092477,1.246863,14.097593


In [40]:
lastdate = basket_p['InvoiceDate'].max().date()

registration = pd.DataFrame(basket_p.groupby(by=['CustomerID'])['InvoiceDate'].min())
purchase     = pd.DataFrame(basket_p.groupby(by=['CustomerID'])['InvoiceDate'].max())

test  = registration.applymap(lambda x:(lastdate - x.date()).days)
test2 = purchase.applymap(lambda x:(lastdate - x.date()).days)

transactions.loc[:, 'LastPurchase'] = test2.reset_index(drop = False)['InvoiceDate']
transactions.loc[:, 'FirstPurchase'] = test.reset_index(drop = False)['InvoiceDate']

transactions[:5]

,CustomerID,count,min,max,mean,sum,categ_0,categ_1,categ_2,categ_3,categ_4,LastPurchase,FirstPurchase
0,12346.0,1,77183.60,77183.60,77183.600000,77183.60,0.000000,0.000000,100.000000,0.000000,0.000000,325,325
1,12347.0,7,224.82,1294.32,615.714286,4310.00,29.540371,25.242459,26.375870,11.237123,7.604176,2,367
2,12348.0,4,227.44,892.80,449.310000,1797.24,0.000000,20.030714,41.953217,38.016069,0.000000,75,358
3,12349.0,1,1757.55,1757.55,1757.550000,1757.55,10.713778,37.877728,26.506216,4.513101,20.389178,18,18
4,12350.0,1,334.40,334.40,334.400000,334.40,0.000000,39.862440,48.444976,11.692584,0.000000,310,310


In [41]:
n_1 = transactions[transactions['count'] == 1].shape[0]
n_2 = transactions.shape[0]
n_1


1495

In [42]:
list_c = ['count','min','max','mean','categ_0','categ_1','categ_2','categ_3','categ_4']

s_customers = transactions.copy(deep = True)
matrix = s_customers[list_c].values

In [43]:
scaler = StandardScaler()
scaler.fit(matrix)
scaled_matrix = scaler.transform(matrix)


pca = PCA()
pca.fit(scaled_matrix)
pca_samples = pca.transform(scaled_matrix)

In [44]:
n_clusters = 10
kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=100)
kmeans.fit(scaled_matrix)
clusters_clients = kmeans.predict(scaled_matrix)
silhouette_avg = silhouette_score(scaled_matrix, clusters_clients)
print('score of silhouette: {:<.3f}'.format(silhouette_avg))

score of silhouette: 0.247


In [45]:
pca = PCA(n_components=6)
matrix_3D = pca.fit_transform(scaled_matrix)
mat = pd.DataFrame(matrix_3D)
mat['cluster'] = pd.Series(clusters_clients)
s_customers.loc[:, 'cluster'] = clusters_clients


In [46]:
merged = pd.DataFrame()
for i in range(n_clusters):
    test = pd.DataFrame(s_customers[s_customers['cluster'] == i].mean())
    test = test.T.set_index('cluster', drop = True)
    test['size'] = s_customers[s_customers['cluster'] == i].shape[0]
    merged = pd.concat([merged, test])

merged.drop('CustomerID', axis = 1, inplace = True)
print('number of customers:', merged['size'].sum())

merged = merged.sort_values('sum')

number of customers: 4338


In [47]:
print('Duplicate Entries: {}'.format(data.duplicated().sum()))
data.drop_duplicates(inplace = True)

Duplicate Entries: 0


In [48]:
liste_id = []
for i in range(5):
    column = 'categ_{}'.format(i)
    liste_id.append(merged[merged[column] > 45].index.values[0])

listeorder = liste_id
listeorder += [ s for s in merged.index if s not in liste_id]

merged = merged.reindex(index = listeorder)
merged = merged.reset_index(drop = False)
display(merged[['cluster', 'count', 'min', 'max', 'mean', 'sum', 'categ_0',
                   'categ_1', 'categ_2', 'categ_3', 'categ_4', 'size']])

,cluster,count,min,max,mean,sum,categ_0,categ_1,categ_2,categ_3,categ_4,size
0,1.0,2.686154,255.678646,442.920338,338.200436,1020.437292,53.080000,16.766585,13.742123,6.763282,9.648010,325
1,6.0,2.746102,266.454410,468.746949,358.011245,1029.503875,6.994516,68.642860,11.160077,5.410098,7.792449,449
2,5.0,2.734694,237.176892,393.221978,309.043143,838.802403,8.970577,18.082705,54.084385,13.230323,5.632010,637
3,9.0,2.494652,218.107995,382.327380,287.295284,801.202888,7.695570,15.267821,20.924268,50.176781,5.935560,374
4,3.0,2.982278,230.753927,407.815063,310.671850,956.288914,12.838907,19.843746,11.268962,5.523213,50.525172,395
5,8.0,3.945246,259.476013,600.551011,408.691785,1566.215422,18.443102,29.527981,24.769495,12.547790,14.711633,2009
6,0.0,26.464286,117.094571,2920.259071,814.188103,21536.988643,17.148173,33.356860,22.758631,10.979850,15.756486,140
7,7.0,128.000000,9.435714,2248.087143,383.779559,50111.057143,14.280876,29.481544,25.128558,13.764915,17.344107,7
8,2.0,1.000000,77183.600000,77183.600000,77183.600000,77183.600000,0.000000,0.000000,100.000000,0.000000,0.000000,1
9,4.0,2.000000,2.900000,168469.600000,84236.250000,168472.500000,0.000000,99.998279,0.001721,0.000000,0.000000,1


In [49]:
class Method(object):
    def __init__(self, clf, params=None):
        if params:            
            self.clf = clf(**params)
        else:
            self.clf = clf()

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def grid_search(self, parameters, Kfold):
        self.grid = GridSearchCV(estimator = self.clf, param_grid = parameters, cv = Kfold)
        
    def grid_fit(self, X, Y):
        self.grid.fit(X, Y)
        
    def grid_predict(self, X, Y):
        self.predictions = self.grid.predict(X)
        print("Accuracy: {:.2f} % ".format(100*metrics.accuracy_score(Y, self.predictions)))

### X_train and X_test

In [50]:
columns = ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4' ]
X = s_customers[columns]
Y = s_customers['cluster']

In [51]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, train_size = 0.8, random_state=123)
X_train.shape, X_test.shape  #check the shape of X_train and X_test


((3470, 6), (868, 6))

### kNN

In [52]:
knn = Method(clf = neighbors.KNeighborsClassifier)
knn.grid_search( parameters = [{'n_neighbors': np.arange(1,50,1)}] , Kfold = 5)
knn.grid_fit(X = X_train, Y = Y_train)
knn.grid_predict(X_test, Y_test)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 83.06 % 


In [53]:
rf = Method(clf = ensemble.RandomForestClassifier)
param_grid = {'criterion' : ['entropy', 'gini'], 'n_estimators' : [20, 40, 60, 80, 100],
               'max_features' :['sqrt', 'log2']}
rf.grid_search(parameters = param_grid, Kfold = 5)
rf.grid_fit(X = X_train, Y = Y_train)
rf.grid_predict(X_test, Y_test)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 94.01 % 


### GradientBoostingClassifier

In [54]:
gb = Method(clf = ensemble.GradientBoostingClassifier)
param_grid = {'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}
gb.grid_search(parameters = param_grid, Kfold = 5)
gb.grid_fit(X = X_train, Y = Y_train)
gb.grid_predict(X_test, Y_test)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 93.66 % 


### SVC

In [55]:
svc = Method(clf = svm.LinearSVC)
svc.grid_search(parameters = [{'C':np.logspace(-2,2,10)}], Kfold = 5)

In [56]:
svc.grid_fit(X = X_train, Y = Y_train)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increa

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sahik\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

In [57]:
 svc.grid_predict(X_test, Y_test)

Accuracy: 83.41 % 


### DecisionTreeClassifier

In [58]:
dtc = Method(clf = tree.DecisionTreeClassifier)
dtc.grid_search(parameters = [{'criterion' : ['entropy', 'gini'], 'max_features' :['sqrt', 'log2']}], Kfold = 5)
dtc.grid_fit(X = X_train, Y = Y_train)
dtcc = dtc.grid_predict(X_test, Y_test)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 88.48 % 


In [59]:
ada = Method(clf = AdaBoostClassifier)
param_grid = {'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}
ada.grid_search(parameters = param_grid, Kfold = 5)
ada.grid_fit(X = X_train, Y = Y_train)
ada.grid_predict(X_test, Y_test)

C:\Users\sahik\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Accuracy: 90.32 % 


In [60]:
rf_best  = ensemble.RandomForestClassifier(**rf.grid.best_params_)
gb_best  = ensemble.GradientBoostingClassifier(**gb.grid.best_params_)
svc_best = svm.LinearSVC(**svc.grid.best_params_)
tr_best  = tree.DecisionTreeClassifier(**tr.grid.best_params_)
knn_best = neighbors.KNeighborsClassifier(**knn.grid.best_params_)
lr_best  = linear_model.LogisticRegression(**lr.grid.best_params_)


NameError: name 'tr' is not defined